# Getting Started

To begin with, cobrapy comes with bundled models for _Salmonella_ and _E. coli_, as well as a "textbook" model of _E. coli_ core metabolism. To load a test model, type

In [1]:
from __future__ import print_function
import cobra.test

# "ecoli" and "salmonella" are also valid arguments
model = cobra.test.create_test_model("textbook")

The reactions, metabolites, and genes attributes of the cobrapy model are a special type of list called a DictList, and each one is made up of Reaction, Metabolite and Gene objects respectively.

In [2]:
print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

95
72
137


Just like a regular list, objects in the DictList can be retrived by index. For example, to get the 30th reaction in the model (at index 29 because of [0-indexing](https://en.wikipedia.org/wiki/Zero-based_numbering)):

In [3]:
model.reactions[29]

<Reaction EX_glu__L_e at 0x7f56b0ea3198>

Addictionally, items can be retrived by their id using the get_by_id() function. For example, to get the cytosolic atp metabolite object (the id is "atp_c"), we can do the following:

In [4]:
model.metabolites.get_by_id("atp_c")

<Metabolite atp_c at 0x7f56b0ed7cc0>

As an added bonus, users with an interactive shell such as IPython will be able to tab-complete to list elements inside a list. While this is not recommended behavior for most code because of the possibility for characters like "-" inside ids, this is very useful while in an interactive prompt:

In [5]:
model.reactions.EX_glc__D_e.lower_bound

-10.0

## Reactions

We will consider the reaction glucose 6-phosphate isomerase, which interconverts glucose 6-phosphate and fructose 6-phosphate. The reaction id for this reaction in our test model is PGI.

In [6]:
pgi = model.reactions.get_by_id("PGI")
pgi

<Reaction PGI at 0x7f56b0e396d8>

We can view the full name and reaction catalyzed as strings

In [7]:
print(pgi.name)
print(pgi.reaction)

glucose-6-phosphate isomerase
g6p_c <=> f6p_c


We can also view reaction upper and lower bounds. Because the pgi.lower_bound < 0, and pgi.upper_bound > 0, pgi is reversible

In [8]:
print(pgi.lower_bound, "< pgi <", pgi.upper_bound)
print(pgi.reversibility)

-1000.0 < pgi < 1000.0
True


We can also ensure the reaction is mass balanced. This function will return elements which violate mass balance. If it comes back empty, then the reaction is mass balanced.

In [9]:
pgi.check_mass_balance()

{}

In order to add a metabolite, we pass in a dict with the metabolite object and its coefficient

In [10]:
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): -1})
pgi.reaction

'g6p_c + h_c <=> f6p_c'

The reaction is no longer mass balanced

In [11]:
pgi.check_mass_balance()

{'H': -1.0, 'charge': -1.0}

We can remove the metabolite, and the reaction will be balanced once again.

In [12]:
pgi.pop(model.metabolites.get_by_id("h_c"))
print(pgi.reaction)
print(pgi.check_mass_balance())

g6p_c <=> f6p_c
{}


It is also possible to build the reaction from a string. However, care must be taken when doing this to ensure reaction id's match those in the model. The direction of the arrow is also used to update the upper and lower bounds.

In [13]:
pgi.reaction = "g6p_c --> f6p_c + h_c + green_eggs + ham"

unknown metabolite 'green_eggs' created
unknown metabolite 'ham' created


In [14]:
pgi.reaction

'g6p_c --> f6p_c + green_eggs + h_c + ham'

In [15]:
pgi.reaction = "g6p_c <=> f6p_c"
pgi.reaction

'g6p_c <=> f6p_c'

## Metabolites

We will consider cytosolic atp as our metabolite, which has the id atp_c in our test model.

In [16]:
atp = model.metabolites.get_by_id("atp_c")
atp

<Metabolite atp_c at 0x7f56b0ed7cc0>

We can print out the metabolite name and compartment (cytosol in this case).

In [17]:
print(atp.name)
print(atp.compartment)

ATP
c


We can see that ATP is a charged molecule in our model.

In [18]:
atp.charge

-4

We can see the chemical formula for the metabolite as well.

In [19]:
print(atp.formula)

C10H12N5O13P3


The reactions attribute gives a frozenset of all reactions using the given metabolite. We can use this to count the number of reactions which use atp.

In [20]:
len(atp.reactions)

13

A metabolite like glucose 6-phosphate will participate in fewer reactions.

In [21]:
model.metabolites.get_by_id("g6p_c").reactions

frozenset({<Reaction GLCpts at 0x7f56b0eabcc0>,
           <Reaction Biomass_Ecoli_core at 0x7f56b0e9e358>,
           <Reaction G6PDH2r at 0x7f56b0eab9e8>,
           <Reaction PGI at 0x7f56b0e396d8>})

## Genes

The gene_reaction_rule is a boolean representation of the gene requirements for this reaction to be active as described in [Schellenberger et al 2011 Nature Protocols 6(9):1290-307](http://dx.doi.org/doi:10.1038/nprot.2011.308).

The GPR is stored as the gene_reaction_rule for a Reaction object as a string.

In [22]:
gpr = pgi.gene_reaction_rule
gpr

'b4025'

Corresponding gene objects also exist. These objects are tracked by the reactions itself, as well as by the model

In [23]:
pgi.genes

frozenset({<Gene b4025 at 0x7f56b0e8fac8>})

In [24]:
pgi_gene = model.genes.get_by_id("b4025")
pgi_gene

<Gene b4025 at 0x7f56b0e8fac8>

Each gene keeps track of the reactions it catalyzes

In [25]:
pgi_gene.reactions

frozenset({<Reaction PGI at 0x7f56b0e396d8>})

Altering the gene_reaction_rule will create new gene objects if necessary and update all relationships.

In [26]:
pgi.gene_reaction_rule = "(spam or eggs)"
pgi.genes

frozenset({<Gene eggs at 0x7f56b0e35ba8>, <Gene spam at 0x7f56b0e390f0>})

In [27]:
pgi_gene.reactions

frozenset()

Newly created genes are also added to the model

In [28]:
model.genes.get_by_id("spam")

<Gene spam at 0x7f56b0e390f0>

The delete_model_genes function will evaluate the gpr and set the upper and lower bounds to 0 if the reaction is knocked out. This function can preserve existing deletions or reset them using the cumulative_deletions flag.

In [29]:
cobra.manipulation.delete_model_genes(model, ["spam"],
                                      cumulative_deletions=True)
print("after 1 KO: %4d < flux_PGI < %4d" %
      (pgi.lower_bound, pgi.upper_bound))

cobra.manipulation.delete_model_genes(model, ["eggs"],
                                      cumulative_deletions=True)
print("after 2 KO:  %4d < flux_PGI < %4d" %
      (pgi.lower_bound, pgi.upper_bound))

after 1 KO: -1000 < flux_PGI < 1000
after 2 KO:     0 < flux_PGI <    0


The undelete_model_genes can be used to reset a gene deletion

In [30]:
cobra.manipulation.undelete_model_genes(model)
print(pgi.lower_bound, "< pgi <", pgi.upper_bound)

-1000 < pgi < 1000
